Goal is to get only city and country into a pandas column to easily group by country or city and country later on.

In [ ]:
import pandas as pd
import requests

In [3]:
#import the nodes 
df = pd.read_csv("osm_nodes_flip.csv", encoding = 'iso-8859-1')

In [4]:
#create desired empty columns
df['geocode_data'] = ''
df['city'] = ''
df['country'] = ''

In [8]:
df.head()

,Unnamed: 0,geometry.coordinates,geometry.type,id,properties.abandoned,properties.addr:city,properties.addr:country,properties.addr:postcode,properties.addr:state,properties.addr:street,...,properties.tourism,properties.units,properties.waterway,properties.website,properties.wikidata,properties.wikipedia,type,geocode_data,city,country
0,0,"[51.0745439, -115.4022874]",Point,30445652,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,,,
1,1,"[43.2439679, -116.3783661]",Point,126161573,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,,,
2,2,"[48.1056465, -1.9880204]",Point,263782394,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,,,
3,3,"[49.6197688, 10.8350521]",Point,318895153,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,,,
4,4,"[43.2669091, -122.4098458]",Point,365796528,NaN,NaN,NaN,NaN,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,,,


Define reverse geocode function using google API to apply to all rows of df

In [6]:
def reverse_geocode(latlng):
    result = {}
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}'
    request = url.format(latlng)
    data = requests.get(request).json()
    if len(data['results']) > 0:
        result = data['results'][0]
    return result

In [11]:
#take out the brackets bc geocoder doesn't work if you don't 
df['geometry.coordinates']=df['geometry.coordinates'].str.replace("[","")
df['geometry.coordinates']=df['geometry.coordinates'].str.replace("]","")

In [12]:
#map the geocoder to all rows using the coordinates column
df['geocode_data'] = df['geometry.coordinates'].map(reverse_geocode)

In [14]:
#quick look
df['geocode_data'][0]

{'address_components': [{'long_name': 'Ken Richie Way',
   'short_name': 'Ken Richie Way',
   'types': ['route']},
  {'long_name': 'Canmore',
   'short_name': 'Canmore',
   'types': ['locality', 'political']},
  {'long_name': 'Division No. 15',
   'short_name': 'Division No. 15',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Alberta',
   'short_name': 'AB',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'Canada',
   'short_name': 'CA',
   'types': ['country', 'political']},
  {'long_name': 'T0L',
   'short_name': 'T0L',
   'types': ['postal_code', 'postal_code_prefix']}],
 'formatted_address': 'Ken Richie Way, Canmore, AB T0L, Canada',
 'geometry': {'bounds': {'northeast': {'lat': 51.0754026, 'lng': -115.4013776},
   'southwest': {'lat': 51.0748264, 'lng': -115.4022129}},
  'location': {'lat': 51.0751365, 'lng': -115.4018363},
  'location_type': 'GEOMETRIC_CENTER',
  'viewport': {'northeast': {'lat': 51.07646348029149,
    'lng'

Define functions to parse out country name and city name!!

In [15]:
def parse_country(geocode_data):
    if (not geocode_data is None) and ('address_components' in geocode_data):
        for component in geocode_data['address_components']:
            if 'country' in component['types']:
                return component['long_name']
    return None
 
def parse_city(geocode_data):
    if (not geocode_data is None) and ('address_components' in geocode_data):
        for component in geocode_data['address_components']:
            if 'locality' in component['types']:
                return component['long_name']
            elif 'postal_town' in component['types']:
                return component['long_name']
            elif 'administrative_area_level_2' in component['types']:
                return component['long_name']
            elif 'administrative_area_level_1' in component['types']:
                return component['long_name']
    return None

In [16]:
df['city'] = df['geocode_data'].map(parse_city)
df['country'] = df['geocode_data'].map(parse_country)

In [17]:
#quick check to makes sure it worked
df.head()

,Unnamed: 0,geometry.coordinates,geometry.type,id,properties.abandoned,properties.addr:city,properties.addr:country,properties.addr:postcode,properties.addr:state,properties.addr:street,...,properties.tourism,properties.units,properties.waterway,properties.website,properties.wikidata,properties.wikipedia,type,geocode_data,city,country
0,0,"51.0745439, -115.4022874",Point,30445652,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,"{'formatted_address': 'Ken Richie Way, Canmore...",Canmore,Canada
1,1,"43.2439679, -116.3783661",Point,126161573,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,"{'formatted_address': 'Swan Falls Dam, Murphy,...",Murphy,United States
2,2,"48.1056465, -1.9880204",Point,263782394,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,"{'formatted_address': 'Le Ricotay, 35750 Iffen...",Iffendic,France
3,3,"49.6197688, 10.8350521",Point,318895153,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,"{'formatted_address': 'Reuther Weg 10, 91085 W...",Weisendorf,Germany
4,4,"43.2669091, -122.4098458",Point,365796528,NaN,NaN,NaN,NaN,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Feature,"{'formatted_address': 'Clearwater Plant Rd, Id...",Douglas County,United States


50 different countries are listed in the nodes df

In [21]:
print(len(df.country.unique()))
df.country.unique()

50


array(['Canada', 'United States', 'France', 'Germany', 'Sierra Leone',
       'Austria', 'Iceland', 'Brazil', 'Morocco', 'Norway', 'Spain',
       'Australia', 'Taiwan', 'Mexico', 'China', 'Ecuador', 'North Korea',
       'South Africa', 'Greece', 'Ethiopia', 'United Kingdom', 'Nepal',
       'Poland', None, 'Japan', 'Libya', 'Togo', 'Vietnam', 'Switzerland',
       'Zambia', 'Haiti', 'Sri Lanka', 'Portugal', 'Uganda', 'Denmark',
       'Kuwait', 'Italy', 'Czech Republic', 'Ukraine', 'Syria',
       'Indonesia', 'Peru', 'Tanzania', 'Iran', 'Kenya', 'Philippines',
       'Myanmar (Burma)', 'Finland', 'India', 'Fiji'], dtype=object)

Do the same with the ways df.  Note that I quickly edited the csv file to only get the first pair of coordinates.  I'm sure there is a pythonic way to do this as well.

In [22]:
df2 = pd.read_csv("osm_ways_flip_2.csv", encoding = 'iso-8859-1')

In [23]:
df2.head()

,id,Unnamed: 1,lat,long,combined
0,4670340,0,51.636868,7.624084,"51.6368677,7.624084"
1,4677631,1,52.538009,13.349289,"52.5380095,13.3492891"
2,4755466,2,52.482635,13.307154,"52.4826347,13.3071541"
3,4763495,3,52.523081,13.313312,"52.523081,13.3133122"
4,4800321,4,54.633256,-1.183778,"54.6332559,-1.183778"


In [24]:
df2['geocode_data'] = ''
df2['city'] = ''
df2['country'] = ''

CAUTION: this takes a long time to run

In [25]:
df2['geocode_data'] = df2['combined'].map(reverse_geocode)

In [28]:
df2.head()

,id,Unnamed: 1,lat,long,combined,geocode_data,city,country
0,4670340,0,51.636868,7.624084,"51.6368677,7.624084","{'formatted_address': 'Unnamed Road, 59192 Ber...",Bergkamen,Germany
1,4677631,1,52.538009,13.349289,"52.5380095,13.3492891","{'formatted_address': 'Mitte, Friedrich-Krause...",Berlin,Germany
2,4755466,2,52.482635,13.307154,"52.4826347,13.3071541",{'formatted_address': 'Bezirk Charlottenburg-W...,Berlin,Germany
3,4763495,3,52.523081,13.313312,"52.523081,13.3133122",{'formatted_address': 'Bezirk Charlottenburg-W...,Berlin,Germany
4,4800321,4,54.633256,-1.183778,"54.6332559,-1.183778","{'formatted_address': 'Unnamed Road, Hartlepoo...",Hartlepool,United Kingdom


In [27]:
df2['city'] = df2['geocode_data'].map(parse_city)
df2['country'] = df2['geocode_data'].map(parse_country)

117 countries in the ways df

In [34]:
print(len(df2['country'].unique()))
df2['country'].unique()

117


array(['Germany', 'United Kingdom', 'France', 'Sweden', 'Netherlands',
       'Poland', 'Belgium', 'Slovakia', 'Switzerland', 'Spain', 'Greece',
       'Malta', 'Italy', 'Austria', 'Canada', 'Finland', 'Russia',
       'Ukraine', 'Malaysia', 'Australia', 'United States', 'Israel',
       'Denmark', 'Belarus', 'Lebanon', 'Cyprus', 'Czech Republic',
       'Tajikistan', 'India', 'Iran', 'Egypt', 'China', 'Chile',
       'Burkina Faso', 'Panama', 'Dominican Republic', 'Brazil', 'Libya',
       'Croatia', 'Reunion', 'Romania', 'Saudi Arabia', 'Nicaragua',
       'Zambia', 'Mexico', None, 'Algeria', 'Kazakhstan',
       'United Arab Emirates', 'Haiti', 'Cuba', 'Turkey',
       'Democratic Republic of the Congo', 'Iceland', 'Iraq', 'Slovenia',
       'South Africa', "Côte d'Ivoire", 'Turkmenistan', 'Indonesia',
       'Tunisia', 'New Zealand', 'Sri Lanka', 'Uzbekistan', 'Japan',
       'Philippines', 'Hungary', 'Nepal', 'Portugal', 'Oman', 'Laos',
       'Namibia', 'Tanzania', 'Costa Rica', 

Save to csv so you don't have to run the reverse geolocator again (it's costly)

In [35]:
df.to_csv('get_country_nodes.csv')
df2.to_csv('get_country_ways.csv')